In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


In [3]:
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [4]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [5]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [6]:
target = df.pop('target')

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [8]:
# Visualize dataset for fitst 5 data vectors

for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


In [9]:
# Shuffle and batch the dataset

train_dataset = dataset.shuffle(len(df)).batch(1)

Passing a dictionary as an input to a model is as easy as creating a matching dictionary of tf.keras.layers.Input layers, applying any pre-processing and stacking them up using the functional api. You can use this as an alternative to feature columns.

In [10]:
inputs = {key: tf.keras.layers.Input(shape = (), name = key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis = -1)

x = tf.keras.layers.Dense(10, activation = 'relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs = inputs, outputs = output)

model_func.compile(optimizer = 'adam',
                   loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics = ['accuracy'])

model_func.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age (InputLayer)                [(None,)]            0                                            
__________________________________________________________________________________________________
sex (InputLayer)                [(None,)]            0                                            
__________________________________________________________________________________________________
cp (InputLayer)                 [(None,)]            0                                            
__________________________________________________________________________________________________
trestbps (InputLayer)           [(None,)]            0                                            
______________________________________________________________________________________________

In [11]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [12]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57],
      dtype=int32)>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3], dtype=int32)>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150], dtype=int32)>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168], dtype=int32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int32)>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0], dtype=int32)>, 'thalach': <tf.Tensor: shape=(16

In [13]:
model_func.fit(dict_slices, epochs = 15)

Train for 19 steps
Epoch 1/15
19/19 [==============================] - 1s 36ms/step - loss: 8.4390 - accuracy: 0.6469
Epoch 2/15
19/19 [==============================] - 0s 6ms/step - loss: 5.8610 - accuracy: 0.4851
Epoch 3/15
19/19 [==============================] - 0s 6ms/step - loss: 4.9564 - accuracy: 0.5743
Epoch 4/15
19/19 [==============================] - 0s 6ms/step - loss: 4.2817 - accuracy: 0.5248
Epoch 5/15
19/19 [==============================] - 0s 7ms/step - loss: 3.7049 - accuracy: 0.5446
Epoch 6/15
19/19 [==============================] - 0s 6ms/step - loss: 3.1982 - accuracy: 0.5512
Epoch 7/15
19/19 [==============================] - 0s 6ms/step - loss: 2.7495 - accuracy: 0.5611
Epoch 8/15
19/19 [==============================] - 0s 7ms/step - loss: 2.3257 - accuracy: 0.5710
Epoch 9/15
19/19 [==============================] - 0s 7ms/step - loss: 1.9320 - accuracy: 0.5908
Epoch 10/15
19/19 [==============================] - 0s 8ms/step - loss: 1.5755 - accuracy: 0.6106